In [1]:
!pip install /Users/johnmount/Documents/work/pyvtreat/pkg/dist/vtreat-0.1.tar.gz
#!pip install https://github.com/WinVector/pyvtreat/raw/master/pkg/dist/vtreat-0.1.tar.gz

Processing /Users/johnmount/Documents/work/pyvtreat/pkg/dist/vtreat-0.1.tar.gz
  Stored in directory: /Users/johnmount/Library/Caches/pip/wheels/cf/06/fc/6b2552717486fb6401f19308eec24381555e456e3bd9cfb103
Successfully built vtreat
  Found existing installation: vtreat 0.1
    Uninstalling vtreat-0.1:
      Successfully uninstalled vtreat-0.1


In [2]:
import pandas
import numpy.random
import vtreat
import sklearn.linear_model
import seaborn
import matplotlib.pyplot
import statsmodels.api
import scipy.stats
import re

In [3]:
n_rows = 2000
n_signal_variables = 2
n_noise_variables = 2
n_levels = 500

In [4]:
d = pandas.DataFrame({"y":numpy.random.normal(size = n_rows)})

In [5]:
def mk_var_values(n_levels):
    values = {}
    for i in range(n_levels):
        values["level_" + str(i)] = numpy.random.normal(size=1)[0]
    return values

In [6]:
for i in range(n_signal_variables):
    var_name = "var_" + str(i)
    levs = mk_var_values(n_levels)
    keys = [ k for k in levs.keys() ]
    observed = [ keys[i] for i in numpy.random.choice(len(keys), size=n_rows, replace=True)]
    effect = numpy.asarray([ levs[k] for k in observed ])
    d[var_name] = observed
    d["y"] = d["y"] + effect

In [7]:
for i in range(n_noise_variables):
    var_name = "noise_" + str(i)
    levs = mk_var_values(n_levels)
    keys = [ k for k in levs.keys() ]
    observed = [ keys[i] for i in numpy.random.choice(len(keys), size=n_rows, replace=True)]
    d[var_name] = observed

In [8]:
d.head()

,y,var_0,var_1,noise_0,noise_1
0,-2.118967,level_156,level_24,level_210,level_298
1,-2.672423,level_425,level_422,level_479,level_124
2,-2.279930,level_351,level_429,level_263,level_199
3,-2.567556,level_230,level_148,level_74,level_129
4,-0.203438,level_297,level_321,level_305,level_162


In [9]:
is_train = numpy.random.uniform(size=n_rows)<=0.5

In [10]:
d_train = d.loc[is_train,:].copy()
d_train.reset_index(inplace=True, drop=True)
y_train = numpy.asarray(d_train["y"])
d_train.drop(["y"], axis=1, inplace=True)
d_test = d.loc[numpy.logical_not(is_train),:].copy()
d_test.reset_index(inplace=True, drop=True)
y_test = numpy.asarray(d_test["y"])
d_test.drop(["y"], axis=1, inplace=True)

In [11]:
plan = vtreat.NumericOutcomeTreatment()

In [12]:
cross_frame = plan.fit_transform(d_train, y_train)
prepared_test = plan.transform(d_test)
naive_train = plan.transform(d_train)

In [13]:
cross_frame.head()

,var_0_impact_code,var_0_prevalence_code,var_1_impact_code,noise_1_deviance_code
0,0.000000,0.000000,1.387779e-17,1.681304
1,-0.259624,0.004065,-1.001398e+00,1.584919
2,0.000000,0.001016,-1.981960e+00,0.663339
3,-0.511586,0.006098,0.000000e+00,0.859777
4,0.169751,0.003049,-5.824780e-01,0.000000


In [14]:
naive_train.head()

,var_0_impact_code,var_0_prevalence_code,var_1_impact_code,noise_1_deviance_code
0,0.000000,0.000000,-2.448471,1.040710
1,-1.143022,0.004065,-1.134938,1.521395
2,-0.567791,0.001016,-1.090240,0.898805
3,-0.584434,0.006098,-0.660154,0.862847
4,-0.145631,0.003049,-1.090240,1.669760


In [15]:
all_vars = [vi for vi in plan.score_frame_["variable"]]
corr_frame = pandas.DataFrame({"variable":[vi for vi in all_vars if re.match(".*_impact_.*", vi)]})
corr_frame["naive_train_correlation"] = [ 
    scipy.stats.pearsonr(naive_train[vi], y_train)[0] for vi in corr_frame["variable"]]
corr_frame["cross_frame_correlation"] = [ 
    scipy.stats.pearsonr(cross_frame[vi], y_train)[0] for vi in corr_frame["variable"]]
corr_frame["test_correlation"] = [ 
    scipy.stats.pearsonr(prepared_test[vi], y_test)[0] for vi in corr_frame["variable"]]
corr_frame["is_noise"] = [re.match("^noise_.*", vi) is not None for vi in corr_frame["variable"]]
corr_frame


KeyError: 'noise_0_impact_code'

In [ ]:
seaborn.scatterplot(x = "naive_train_correlation", y = "test_correlation", data = corr_frame, hue = "is_noise")
matplotlib.pyplot.plot([-1, 1], [-1, 1], color="red")
matplotlib.pyplot.xlim(-.2,1)
matplotlib.pyplot.ylim(-.2,1)
matplotlib.pyplot.gca().set_aspect('equal', adjustable='box')

In [ ]:
seaborn.scatterplot(x = "cross_frame_correlation", y = "test_correlation", data = corr_frame,  hue = "is_noise")
matplotlib.pyplot.plot([-1, 1], [-1, 1], color="red")
matplotlib.pyplot.xlim(-.2,1)
matplotlib.pyplot.ylim(-.2,1)
matplotlib.pyplot.gca().set_aspect('equal', adjustable='box')

In [ ]:
plan.score_frame_.tail()

In [ ]:
recommended_vars = [vi for vi in plan.score_frame_["variable"][plan.score_frame_["recommended"]]]
recommended_vars

In [ ]:
plot_train = pandas.DataFrame({"y":y_train})
plot_test = pandas.DataFrame({"y":y_test})

In [ ]:
fitter = sklearn.linear_model.LinearRegression(fit_intercept = True)

In [ ]:
fitter.fit(cross_frame[all_vars], y_train)
plot_train["predict_cross_all_vars"] = fitter.predict(cross_frame[all_vars])
plot_test["predict_cross_all_vars"] = fitter.predict(prepared_test[all_vars])

In [ ]:
fitter.fit(cross_frame[recommended_vars], y_train)
plot_train["predict_cross_recommended_vars"] = fitter.predict(cross_frame[recommended_vars])
plot_test["predict_cross_recommended_vars"] = fitter.predict(prepared_test[recommended_vars])

In [ ]:
fitter.fit(naive_train[all_vars], y_train)
plot_train["predict_naive_all_vars"] = fitter.predict(naive_train[all_vars])
plot_test["predict_naive_all_vars"] = fitter.predict(prepared_test[all_vars])

In [ ]:
plot_test.head()

In [ ]:
def rmse(x, y):
    return numpy.sqrt(numpy.mean((x-y)**2))

In [ ]:
print(rmse(plot_train["predict_naive_all_vars"], plot_train["y"]))
seaborn.scatterplot(x="predict_naive_all_vars", y ="y", data = plot_train)
plt = matplotlib.pyplot.title("Naive prediction on train")

In [ ]:
print(rmse(plot_train["predict_cross_all_vars"], plot_train["y"]))
seaborn.scatterplot(x="predict_cross_all_vars", y ="y", data = plot_train)
plt = matplotlib.pyplot.title("Cross prediction on (all vars) train")

In [ ]:
print(rmse(plot_train["predict_cross_recommended_vars"], plot_train["y"]))
seaborn.scatterplot(x="predict_cross_recommended_vars", y ="y", data = plot_train)
plt = matplotlib.pyplot.title("Cross prediction (recommended vars) on train")

In [ ]:
print(rmse(plot_test["predict_naive_all_vars"], plot_test["y"]))
print()
seaborn.scatterplot(x="predict_naive_all_vars", y ="y", data = plot_test)
plt = matplotlib.pyplot.title("Naive prediction on test")

In [ ]:
print(rmse(plot_test["predict_cross_all_vars"], plot_test["y"]))
seaborn.scatterplot(x="predict_cross_all_vars", y ="y", data = plot_test)
plt = matplotlib.pyplot.title("Cross prediction on (all vars) test")

In [ ]:
print(rmse(plot_test["predict_cross_recommended_vars"], plot_test["y"]))
seaborn.scatterplot(x="predict_cross_recommended_vars", y ="y", data = plot_test)
plt = matplotlib.pyplot.title("Cross prediction (recommended vars) on test")

In [ ]:
smf = statsmodels.api.OLS(
    y_train, 
    statsmodels.api.add_constant(naive_train[all_vars])).fit()
smf.summary()

In [ ]:
smf = statsmodels.api.OLS(
    y_train, 
    statsmodels.api.add_constant(cross_frame[all_vars])).fit()
smf.summary()

In [ ]:
params = {'use_hierarchical_estimate':True}
params